### Import Libraries

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import pytz

### Read the Excel file

In [2]:
data = pd.read_excel('test 2021-05-06 start.xlsx')

# Handling the missing timestamps

### Create a function which converts string input to datetime format and checks if DST starts or ends

In [3]:
def str_to_datetime(stamp):
    given_tz = pytz.timezone('US/Central')
    try: # If timezone is behind GMT
        # Split and create a timestamp
        stamp_split = stamp.split('-')
        timez = stamp.split()[1].split('-')[1].split(':')
        timez = ''.join(timez)
        stamp_split[-1] = timez
        stamp = '-'.join(stamp_split)
        date_time_input = datetime.strptime(stamp, '%Y-%m-%d %H:%M:%S%z') # Timestamp of input string created

        # To check if DST is applied
        stamp_split.pop()
        stamp = '-'.join(stamp_split)
        date_time_localized = given_tz.localize(datetime.strptime(stamp, '%Y-%m-%d %H:%M:%S')) # Timestamp with the localized timezone information

    except: # If timezone is ahead of GMT
        # Split and create a timestamp
        stamp_split = stamp.split('+')
        timez = stamp_split[1].split(':')
        timez = ''.join(timez)
        stamp_split[-1] = timez
        stamp = '+'.join(stamp_split)
        date_time_input = datetime.strptime(stamp, '%Y-%m-%d %H:%M:%S%z') # Timestamp of input string created

        # To check if DST is applied
        stamp_split.pop()
        date_time_localized = given_tz.localize(datetime.strptime(stamp_split[0], '%Y-%m-%d %H:%M:%S')) # Timestamp with the localized timezone information
    
    # Check to see if DST is applied
    if date_time_input < date_time_localized: # If given time is less than the localized time
        dst = 'off'
    elif date_time_input > date_time_localized: # If given time is greater than the localized time
        dst = 'on'
    else: # If given time is equal to the localized time
        dst = None

    return date_time_localized, dst


### Updating the dataframe to add the missing timestamps

In [4]:
missing_index = [] # Create a list to keep track of the indexes of all the missing timestamps
iter = data['id'][len(data) - 1] # Last index entry
for i in range(1, iter+1):

    curr_id = data['id'][i-1]
    next_id = data['id'][i]

    if curr_id + 1 != next_id: # Check if there is a missing index
        current_time, dst_info = str_to_datetime(data['time'][i-1])
        next_time_check, next_dst = str_to_datetime(str(current_time + timedelta(hours=1)))
        
        if next_dst == 'on': # Condition to check if DST starts
            current_time = str_to_datetime(str(current_time + timedelta(hours=1)))[0] # Lose one hour and return the localized time

        if dst_info == 'off': # Condition to check if DST ends
            next_time = current_time # Gain one hour

        else: # If DST information does not change
            next_time = current_time + timedelta(hours=1)
        
        # Update the dataframe to add a new row
        entry = pd.DataFrame({'id':[int(data['id'][i-1]) + 1],\
            'time': [str(next_time)], 'VTWS_AVG': [None], \
                'WIND_SPEED':[None],'WIND_SPEED_2': [None],\
                    'windspeed5': [None], 'Windspeed3a':[None]})
        data = pd.concat([(data.iloc[:i]), entry, data.iloc[i:]]).reset_index(drop=True)

        missing_index.append(i) # Update the list of indexes of all the missing timestamps

### Create a column to see where the Timestamps are missing

In [5]:
missing_timestamp_list = [] # Create an empty list
iter = data['id'][len(data) - 1] # Last index entry
count_missing_timestamp = 0

# Check if the index is in the missing_index list
for i in range(0, iter+1):
    if i in missing_index:
        missing_timestamp_list.append('Missing from original input dataset') # Appends a string 'Missing from original input dataset' to the list missing_timestamp_list where the index is in the missing_index list
        count_missing_timestamp += 1
    else:
        missing_timestamp_list.append(None) # Appends a null value to the list missing_timestamp_list where the index is not in the missing_index list

data['timestamp flag'] = missing_timestamp_list # Create a new column in the dataframe showing a flag where the timestamp is missing using the list

print("Number of missing timestamps in the original input dataset = ", count_missing_timestamp)

Number of missing timestamps in the original input dataset =  7


# Handling the missing VTWS_AVG values

### Create a column to see where the VTWS_AVG values are missing

In [6]:
vtws_avg_list = [] # Create an empty list
count_missing_VTWS = 0

# Checks which values are null in the VTWS_AVG column
for i in list(data['VTWS_AVG'].isnull()):
    if i == False:
        vtws_avg_list.append(None) # Appends a null value to the list where the VTWS_AVG value is not missing
    else:
        vtws_avg_list.append('Erroneous') # Appends a string 'Erroneous' to the list where the VTWS_AVG value is missing
        count_missing_VTWS += 1

data['data qc flag VTWS_AVG'] = vtws_avg_list # Create a new column in the dataframe showing a flag where the value is missing using the list

print("Number of missing VTWS_AVG values in the original input dataset = ", count_missing_VTWS)

Number of missing VTWS_AVG values in the original input dataset =  45


# Printing the data and creating an excel file

In [7]:
data

,id,time,VTWS_AVG,WIND_SPEED,WIND_SPEED_2,windspeed5,Windspeed3a,timestamp flag,data qc flag VTWS_AVG
0,0,2020-10-01 00:00:00-05:00,10.775331,10.687425,10.728533,11.908320,10.730430,None,None
1,1,2020-10-01 01:00:00-05:00,9.092214,9.165928,9.199237,9.464774,9.152460,None,None
2,2,2020-10-01 02:00:00-05:00,7.908880,8.012490,8.013744,9.959714,7.978371,None,None
3,3,2020-10-01 03:00:00-05:00,7.425524,7.442371,7.475331,7.747372,7.447742,None,None
4,4,2020-10-01 04:00:00-05:00,7.619581,7.656880,7.683305,7.866113,7.653255,None,None
...,...,...,...,...,...,...,...,...,...
1104,1104,2020-11-15 23:00:00-06:00,5.834029,5.828798,6.317920,9.141444,5.993583,None,None
1105,1105,2020-11-16 00:00:00-06:00,6.429171,6.447155,6.925761,10.177430,6.600696,None,None
1106,1106,2020-11-16 01:00:00-06:00,7.299750,7.307296,7.803329,9.154391,7.470125,None,None
1107,1107,2020-11-16 02:00:00-06:00,7.335454,7.430944,8.116082,10.575292,7.627493,None,None


In [8]:
data.to_excel('Result.xlsx', index = False) # export the dataframe to excel